<a href="https://colab.research.google.com/github/FunNyLuAz/Performance-em-Sistemas-Ciberfisicos/blob/main/Atividade%20Pr%C3%A1tica%201%20-%20Caches%20-%202023-1/Atividade%20Pr%C3%A1tica%201%20-%20Caches%20-%202023-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração inicial do ambiente

Variáveis e funções bases para o código.

In [2]:
DEFAULT_FOLDER = "files" # Pasta padrão para os arquivos na máquina virtual
FILE_NAME = "arquivo" # Nome base dos arquivos
FILE_EXT = "txt" # Extensão base dos arquivos
FILE_QUANTITY = 100 # Quantidade total dos arquivos

MAX_CACHE_SIZE = 10 # Tamanho máximo do cache

# Retorna o nome do arquivo de acordo com o índice
def createFileName(index):
   return f"{FILE_NAME}{index:03}.{FILE_EXT}"

Baixando os arquivos de texto para a máquina virtual a partir do meu GitHub.

In [3]:
import requests
import os


# Path para os arquivos salvos no meu GitHub
PATH = "https://raw.githubusercontent.com/FunNyLuAz/Performance-em-Sistemas-Ciberfisicos/main/Atividade%20Pr%C3%A1tica%201%20-%20Caches%20-%202023-1/1000_words_files/"

if os.path.basename(os.getcwd()) != DEFAULT_FOLDER:
  folderPath = os.path.join(os.getcwd(), DEFAULT_FOLDER)
  if not os.path.exists(folderPath):
    os.makedirs(DEFAULT_FOLDER)
  os.chdir(folderPath)

print("\nBaixando arquivos do GitHub...")
for i in range(FILE_QUANTITY):
  curFileName = createFileName(i + 1)
  with open(curFileName, "wb") as file:
    file.write(requests.get(PATH + curFileName, allow_redirects=True).content)
    file.close()
  print(f"{curFileName} baixado.")
print("\nArquivos baixados com sucesso.")


Baixando arquivos do GitHub...
arquivo001.txt baixado.
arquivo002.txt baixado.
arquivo003.txt baixado.
arquivo004.txt baixado.
arquivo005.txt baixado.
arquivo006.txt baixado.
arquivo007.txt baixado.
arquivo008.txt baixado.
arquivo009.txt baixado.
arquivo010.txt baixado.
arquivo011.txt baixado.
arquivo012.txt baixado.
arquivo013.txt baixado.
arquivo014.txt baixado.
arquivo015.txt baixado.
arquivo016.txt baixado.
arquivo017.txt baixado.
arquivo018.txt baixado.
arquivo019.txt baixado.
arquivo020.txt baixado.
arquivo021.txt baixado.
arquivo022.txt baixado.
arquivo023.txt baixado.
arquivo024.txt baixado.
arquivo025.txt baixado.
arquivo026.txt baixado.
arquivo027.txt baixado.
arquivo028.txt baixado.
arquivo029.txt baixado.
arquivo030.txt baixado.
arquivo031.txt baixado.
arquivo032.txt baixado.
arquivo033.txt baixado.
arquivo034.txt baixado.
arquivo035.txt baixado.
arquivo036.txt baixado.
arquivo037.txt baixado.
arquivo038.txt baixado.
arquivo039.txt baixado.
arquivo040.txt baixado.
arquivo0

#Código para LRU (Least Recently Used)

In [4]:
from collections import deque


LRU_cacheHashTable = dict()  # Planilha para associar o número dos arquivos com o índice no cache
LRU_cache = deque()  # Cache do programa


# Reinicializa as variáveis do cache
def LRU_restartCache():
  global LRU_cacheHashTable, LRU_cache
  LRU_cacheHashTable.clear()
  LRU_cache.clear()


# Retorna se o arquivo está no cache
def LRU_isFileInCache(index):
  if index in LRU_cacheHashTable.keys():  # CACHE HIT
    return True
  else:  # CACHE MISS
    return False


# Retorna o arquivo no cache
def LRU_getFileInCache(index):
  i = LRU_cacheHashTable[index]

  LRU_cache.appendleft(LRU_cache[i])
  LRU_cache.rotate(-(i + 1))
  LRU_cache.popleft()
  LRU_cache.rotate((i + 1))

  for key, value in LRU_cacheHashTable.items():
    if value < i:
      LRU_cacheHashTable[key] += 1

  LRU_cacheHashTable[index] = 0

  return LRU_cache[0]


# Adiciona o arquivo ao cache
def LRU_addFileInCache(index, file):
  if len(LRU_cache) >= MAX_CACHE_SIZE:
    i = list(LRU_cacheHashTable.values()).index(len(LRU_cache) - 1)
    LRU_cacheHashTable.pop(list(LRU_cacheHashTable.keys())[i])
    LRU_cache.pop()

  LRU_cache.appendleft(file)

  for key in LRU_cacheHashTable.keys():
    LRU_cacheHashTable[key] += 1

  LRU_cacheHashTable[index] = 0


# Retorna o arquivo diretamente do cache ou resgata ele do disco
def LRU_getFile(index):
  curFileName = createFileName(index)
  text = ""
  isCacheMiss = False
  if not LRU_isFileInCache(index):
    with open(curFileName, "r", encoding="UTF-8") as file:
      text = file.read()
      file.close()
    LRU_addFileInCache(index, text)
    isCacheMiss = True
  else:
    text = LRU_getFileInCache(index)
  return text, isCacheMiss

# Código para LFU (Least Frequently Used)

In [5]:
LFU_cacheFreqTable = dict()  # Planilha para associar o número dos arquivos com a frequência deles
LFU_cache = dict()  # Cache do programa


# Reinicializa as variáveis do cache
def LFU_restartCache():
  global LFU_cacheFreqTable, LFU_cache
  LFU_cacheFreqTable = dict()
  LFU_cache = dict()


# Retorna se o arquivo está no cache
def LFU_isFileInCache(index):
  if index in LFU_cache.keys():  # CACHE HIT
    return True
  else:  # CACHE MISS
    return False


# Retorna o arquivo no cache
def LFU_getFileInCache(index):
  LFU_cacheFreqTable[index] += 1
  return LFU_cache[index]


# Adiciona o arquivo ao cache
def LFU_addFileInCache(index, file):
  if len(LFU_cache) >= MAX_CACHE_SIZE:
    m = min(LFU_cacheFreqTable.values())
    for k, v in LFU_cacheFreqTable.items():
      if v == m:
        LFU_cacheFreqTable.pop(k)
        LFU_cache.pop(k)
        break

  LFU_cacheFreqTable[index] = 1
  LFU_cache[index] = file


# Retorna o arquivo diretamente do cache ou resgata ele do disco
def LFU_getFile(index, debug=False):
  curFileName = createFileName(index)
  text = ""
  isCacheMiss = False
  if not LFU_isFileInCache(index):
    with open(curFileName, "r", encoding="UTF-8") as file:
      text = file.read()
      file.close()
    LFU_addFileInCache(index, text)
    isCacheMiss = True
  else:
    text = LFU_getFileInCache(index)
  return text, isCacheMiss

# Código para FIFO (First In First Out)



In [6]:
from queue import Queue


FIFO_cacheIndex = Queue(MAX_CACHE_SIZE)  # Fila com índices em ordem da fila do cache
FIFO_cache = Queue(MAX_CACHE_SIZE)  # Cache do programa


# Reinicializa as variáveis do cache
def FIFO_restartCache():
  global FIFO_cacheIndex, FIFO_cache
  FIFO_cacheIndex = Queue(MAX_CACHE_SIZE)
  FIFO_cache = Queue(MAX_CACHE_SIZE)


# Retorna se o arquivo está no cache
def FIFO_isFileInCache(index):
  isFileInCache = False  # CACHE MISS
  cacheIndexTemp = Queue(MAX_CACHE_SIZE)

  while not FIFO_cacheIndex.empty():
    itemKey = FIFO_cacheIndex.get()
    cacheIndexTemp.put(itemKey)
    if itemKey == index:  # CACHE HIT
      isFileInCache = True
  
  while not cacheIndexTemp.empty():
    FIFO_cacheIndex.put(cacheIndexTemp.get())
  
  return isFileInCache


# Retorna o arquivo no cache
def FIFO_getFileInCache(index):
  file = ""
  cacheIndexTemp = Queue(MAX_CACHE_SIZE)
  cacheTemp = Queue(MAX_CACHE_SIZE)

  while not FIFO_cacheIndex.empty():
    itemKey = FIFO_cacheIndex.get()
    itemValue = FIFO_cache.get()
    if itemKey == index:
      file = itemValue
    else:
      cacheIndexTemp.put(itemKey)
      cacheTemp.put(itemValue)

  FIFO_cacheIndex.put(index)
  FIFO_cache.put(file)
  while not cacheIndexTemp.empty():
    FIFO_cacheIndex.put(cacheIndexTemp.get())
    FIFO_cache.put(cacheTemp.get())

  return file


# Adiciona o arquivo ao cache
def FIFO_addFileInCache(index, file):
  cacheIndexTemp = Queue(MAX_CACHE_SIZE)
  cacheTemp = Queue(MAX_CACHE_SIZE)

  while not FIFO_cacheIndex.empty():
    itemKey = FIFO_cacheIndex.get()
    itemValue = FIFO_cache.get()
    if not cacheIndexTemp.full():
      cacheIndexTemp.put(itemKey)
      cacheTemp.put(itemValue)

  FIFO_cacheIndex.put(index)
  FIFO_cache.put(file)
  while not cacheIndexTemp.empty() and not FIFO_cacheIndex.full():
    FIFO_cacheIndex.put(cacheIndexTemp.get())
    FIFO_cache.put(cacheTemp.get())


# Retorna o arquivo diretamente do cache ou resgata ele do disco
def FIFO_getFile(index):
  curFileName = createFileName(index)
  text = ""
  isCacheMiss = False
  if not FIFO_isFileInCache(index):
    with open(curFileName, "r", encoding="UTF-8") as file:
      text = file.read()
      file.close()
    FIFO_addFileInCache(index, text)
    isCacheMiss = True
  else:
    text = FIFO_getFileInCache(index)
  return text, isCacheMiss

# Função do Modo de Simulação

In [7]:
from numpy.random import poisson
from random import randint, random
from time import time_ns

# Modo de simulação dos algoritimos de cache com três usuários
def modoSimulacao():
  totalTries = 200
  outputFileName = "output.txt"

  users = list()  # Lista com todas as tentativas de cada usuário fictício
  users.append([randint(1, FILE_QUANTITY) for _ in range(totalTries)])  # Aleatórios puros
  users.append([int(poisson(FILE_QUANTITY / 2)) for _ in range(totalTries)])  # Aleatórios com distribuição de Poisson
  users.append(list())  # Aleatórios com peso para o intervalo de 30 a 40 de 33%
  for _ in range(totalTries):
    if random() > 1/3:
      n = randint(1, FILE_QUANTITY - 11)
      if n >= 30:
        n += 11  # São onze valores bloqueados (30, 31, ..., 40)
      users[len(users) - 1].append(n)
    else:
      users[len(users) - 1].append(randint(30, 40))

  getFileMethods = [  # Lista para permitir iterar pelos métodos de cada cache
    (LRU_restartCache, LRU_getFile),
    (LFU_restartCache, LFU_getFile),
    (FIFO_restartCache, FIFO_getFile),
  ]

  reportsCaches = list()  # Lista com as informações de cada acesso a um arquivo de cada usuário para cada cache


  # SIMULAÇÃO
  for c in range(len(getFileMethods)): # Para cada tipo de cache
    getFileMethods[c][0]()  # Reinicia a memória de cache
    reportsCaches.append(list())

    for u in range(len(users)):  # Para cada tipo de usuário
      reportsCaches[c].append(list())
      meanDeltaTime = 0  # Média de latência
      amountCacheMiss = 0  # Quantidade de cache miss

      for i in range(totalTries):  # Para cada tentativa do usuário
        userTry = users[u][i]

        initialTime = time_ns()
        _, isCacheMiss = getFileMethods[c][1](userTry)
        endTime = time_ns()

        if isCacheMiss == True:
          amountCacheMiss += 1
        meanDeltaTime += endTime - initialTime
      
      meanDeltaTime /= totalTries

      reportsCaches[c][u].extend([meanDeltaTime, amountCacheMiss, totalTries - amountCacheMiss])  # Dados do usuário no cache específico


  # OUTPUT
  cacheNames = ["LRU", "LFU", "FIFO"]
  usersNames = ["Aleatórios puro", "Aleatórios com distribuição de Poisson", "Aleatórios com peso para o intervalo de 30 a 40 de 33%"]

  text = ""
  for c, cache in enumerate(reportsCaches):
    text += f"{cacheNames[c]}\n"
    for u, user in enumerate(cache):
      text += f"\tUser {u + 1} - {usersNames[u]}\n"
      text += f"\t\tMédia de latência: {user[0]}ns\n"
      text += f"\t\tQuantidade de cache miss: {user[1]}\n"
      text += f"\t\tQuantidade de cache hit: {user[2]}\n\n"
    text += "\n"
    
  with open("../" + outputFileName, "w", encoding="UTF-8") as file:
    file.write(text)
    file.close()

# Programa principal

In [9]:
# Imprime arquivo solicitado na tela
def printTextFile(index, text):
  curFileName = createFileName(index)
  print(f"\n###{curFileName.upper()}###\n" + text + f"\n###{'#' * len(curFileName)}###")


# Seleção do cache principal
while True:
  print("""Selecione a opção de cache principal:
1 - LRU (Least Recently Used)
2 - LFU (Least Frequently Used)
3 - FIFO (First In First Out)
""")
  mainCacheOption = input(" > ")  # 1 - LRU | 2 - LFU | 3 - FIFO
  if mainCacheOption.isdigit():
    mainCacheOption = int(mainCacheOption)
    break

getFile = None
if mainCacheOption == 1:  # Seleciona a função principal para o cache de acordo com sua escolha
  getFile = LRU_getFile
elif mainCacheOption == 2:
  getFile = LFU_getFile
elif mainCacheOption == 3:
  getFile = FIFO_getFile
else:
  print("Não especificado algoritmo de cache.")
  exit()

# Loop da UI
print("\n\nIniciando programa...\n\nPrograma de solicitação de textos com cache\n")
while True:
  msg = input(" > ")

  if msg.lstrip('-').isdigit():
    msg = int(msg)
    if msg == 0:
      print("Programa encerrado.")
      break
    elif msg == -1:
      print("Modo de simulação iniciado...")
      modoSimulacao()
    elif 1 <= msg <= FILE_QUANTITY:
      print(f"Solicitando {createFileName(msg)}...")
      text, _ = getFile(msg)
      printTextFile(msg, text)
    else:
      print("Valor inválido.")
  else:
    print("Entrada inválida.")
  print()

Selecione a opção de cache principal:
1 - LRU (Least Recently Used)
2 - LFU (Least Frequently Used)
3 - FIFO (First In First Out)

 > 1


Iniciando programa...

Programa de solicitação de textos com cache

 > -1
Modo de simulação iniciado...

 > 0
Programa encerrado.
